Basic Urchin Setup:

In [ ]:
#If using urchin for the first time:
pip install oursin

In [1]:
import oursin as urchin
urchin.setup()
import numpy as np
import pandas as pd
import math
#path = 'INSERT PATH OF FILE HERE'
path = '/Users/jasmineschoch/Documents/UW/VBL/Urchin/Examples/data/jasmine/'

(URN) connected to server
Login sent with ID: root


In [70]:
urchin.clear()

Loading Brain Regions:

In [71]:
#Generating overall brain image
regions = pd.read_csv(path + 'region_colors.csv')
regions.head()
areas = regions.loc[:,"Areas"]
colors = regions.loc[:,"Color"]
urchin.ccf.set_visibility({'grey':True})
urchin.ccf.set_material({'grey':'transparent-unlit'})
urchin.ccf.set_alpha({'grey':0.1})

In [6]:
#Generating specific regions
area_colors = {}
area_vis = {}
area_material = {}
for i in range(len(areas)):
  area = areas[i] + "-lh"
  area_vis[area] = True
  area_material = 'opaque-unlit'
  area_colors[area] = (colors[i])
  area = areas[i] + "-rh"
  area_vis[area] = True
  area_material = 'opaque-unlit'
  area_colors[area] = (colors[i])
urchin.ccf.set_visibility(area_vis)
urchin.ccf.set_material(area_material) 
urchin.ccf.set_color(area_colors)

Loading probes:

In [76]:
probe_data = pd.read_csv(path + 'probe_insertions.csv')
probe_data.head()

,entry_point_rl,entry_point_ap,vertical_angle,horizontal_angle,axial_angle,distance_advanced,color,session
0,-640,1500,81.400131,172.959797,0,3800,red,1
1,-3180,-4180,74.096449,322.485452,0,3900,red,1
2,-3800,-3200,72.486409,324.053181,0,3750,red,2
3,-1800,-3500,81.772533,158.999745,0,3750,red,2
4,-1960,-2260,75.936735,152.451569,0,3800,red,3


In [113]:
probe_data["ccf_entry_ap"]= probe_data["entry_point_ap"] +5400
probe_data["ccf_entry_ml"]= probe_data["entry_point_rl"] +5700
probe_data["ccf_entry_dv"]= 330
probe_data["theta"]= np.deg2rad(90-probe_data['vertical_angle'])
probe_data["phi"]= np.deg2rad(probe_data['horizontal_angle'])
probe_data["y"] = probe_data["distance_advanced"] * np.sin(probe_data["theta"])
probe_data["dv"] = 330 + (probe_data["distance_advanced"] * np.cos(probe_data["theta"]))
probe_data["ap"] = probe_data["ccf_entry_ap"] - (probe_data["y"] * np.sin(probe_data["phi"]))
probe_data["ml"] = probe_data["ccf_entry_ml"] + (probe_data["y"] * np.cos(probe_data["phi"]))
for i in range(len(probe_data)):
  color = probe_data.loc[i,"color"]
  if color == "red":
    probe_data.loc[i,"color"] = "#EE204D"
  elif color == "orange":
    probe_data.loc[i,"color"] = "#FF7538" 
  elif color == "light green":
    probe_data.loc[i,"color"] = "#9ACD32"
probe_data["theta"]= np.rad2deg(probe_data['theta'])
probe_data["phi"]= np.rad2deg(probe_data['phi'])
probe_data.head()

,entry_point_rl,entry_point_ap,vertical_angle,horizontal_angle,axial_angle,distance_advanced,color,session,ccf_entry_ap,ccf_entry_ml,ccf_entry_dv,theta,phi,y,dv,ap,ml
0,-640,1500,81.400131,172.959797,0,3800,#EE204D,1,6900,5060,330,8.599869,97.040203,568.225693,4087.275550,6336.058504,4990.354988
1,-3180,-4180,74.096449,322.485452,0,3900,#EE204D,1,1220,2520,330,15.903551,-52.485452,1068.673424,4080.724878,2067.670419,3170.782413
2,-3800,-3200,72.486409,324.053181,0,3750,#EE204D,2,2200,1900,330,17.513591,-54.053181,1128.495105,3906.170969,3113.587000,2562.465091
3,-1800,-3500,81.772533,158.999745,0,3750,#EE204D,2,1900,3900,330,8.227467,111.000255,536.637782,4041.404032,1399.006325,3707.683990
4,-1960,-2260,75.936735,152.451569,0,3800,#EE204D,3,3140,3740,330,14.063265,117.548431,923.373919,4016.106429,2321.318021,3312.941208


In [72]:
#Creating probes
probes = []
for i in range(len(probe_data)):
    probes.append(str(i))
urchin.probes.create(probes)

In [79]:
#Setting probe colors
probe_colors = {}
for probe in probes:
  probe_colors[probe] = probe_data.loc[int(probe),"color"]
urchin.probes.set_color(probe_colors)

In [114]:
#Setting probe positions
probe_positions = {}
for probe in probes:
    probe_positions[probe] = [probe_data.loc[int(probe), "ml"], probe_data.loc[int(probe), "ap"], probe_data.loc[int(probe), "dv"]]
urchin.probes.set_position(probe_positions)

In [115]:
#Setting probe angles
probe_angles = {}
for probe in probes:
    probe_angles[probe] = [probe_data.loc[int(probe), "phi"], probe_data.loc[int(probe), "theta"], 0]
urchin.probes.set_angle(probe_angles)


In [116]:
print(probe_data['ml'].min())


978.7035415955754
